This is a vectorized version of the environment. How does it work? Simply put, instead of running a single environment, we're running them into batches (vectorization). At each time step, instead of formulating a single action $a$, we'll define it as a vector $a=[0, .., n_\text{envs}]$ where each entry corresponds to the action to be performed for an environment.

Thus, if you run 4 simulatenous environments, your observations space becomes (4, num_observations). Because we use a step size of 10 and have a total of 9 snesors, each observation will result into a space of (40,9) elements. The core idea is to speed up the inference and training time of the model instead of querying a single environment.

In our implementation, based on what model you decide, you should take as inputs the observations reshaped to your liking, and predict the actions $a$ from your policy.

In [1]:
import numpy as np
from student_client.student_gym_env_vectorized import create_student_gym_env_vectorized

env = create_student_gym_env_vectorized(
            num_envs=4,
            step_size=10,
            user_token='vec1'
        )

2026-02-11 16:05:54,187 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/session/create "HTTP/1.1 200 OK"
2026-02-11 16:05:54,187 - student_client.student_gym_env_vectorized - INFO - Created new session: 743daa44-97a0-4a64-8e36-7bfbbca8dc05
2026-02-11 16:05:55,123 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/episode/create "HTTP/1.1 200 OK"
2026-02-11 16:05:55,124 - student_client.student_gym_env_vectorized - INFO - Created new episode 1/4: 9e8879d2-ea1f-4948-ba8f-ee50b714c1e3
2026-02-11 16:05:56,109 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/episode/create "HTTP/1.1 200 OK"
2026-02-11 16:05:56,110 - student_client.student_gym_env_vectorized - INFO - Created new episode 2/4: 7679a69e-cf04-4ea9-9352-0c820fc45e48
2026-02-11 16:05:57,054 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/episode/create "HTTP/1.1 200 OK"
2026-02-11 16:05:57,055 - student_client.student_gym_env_vectorized - INFO - Created new episode 3/

In [2]:
print(f"Environment created with {env.num_envs} parallel environments")
print(f"   Episode IDs: {env.episode_ids}")

# Reset all environments
print(f"\n🔄 Resetting all environments...")
observations, infos = env.reset()

print(f"   Observations shape: {observations.shape}")
print(f"   First observation: {observations[0]}")

Environment created with 4 parallel environments
   Episode IDs: ['9e8879d2-ea1f-4948-ba8f-ee50b714c1e3', '7679a69e-cf04-4ea9-9352-0c820fc45e48', 'd55e6dee-95e2-4de6-93c4-23dba6f00b7f', '9ccb01d2-a4a7-4ffa-9b15-dd2925f6e16e']

🔄 Resetting all environments...


2026-02-11 16:06:05,267 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/episode/vectorized_reset "HTTP/1.1 200 OK"
2026-02-11 16:06:05,268 - student_client.student_gym_env_vectorized - INFO - All 4 environments reset successfully


   Observations shape: (4, 9)
   First observation: [7.9689185e+02 1.9379898e+04 3.3612424e+02 1.1213402e+03 3.7234056e-01
 1.3652785e+06 3.9597166e+03 0.0000000e+00 1.0130379e+01]


## Training / Iterations

Here you can iterate through the vectorized environments. You'll notice that actions are a vector where each entry corresponds to the associated environment in the vector.

In A), we automatically reset the envs that have terminated so you can continue for an indefinite amount of steps. As environments don't have the same length, they stop at different times, this helps you reset terminated episodes on the fly.

Tips:
- The step_size return many observations, should you feed each one-by-one in your model, or the full step_size=10 one? The choice is yours!
- There exists multiple ways of exploring the dataset

In [3]:
for step in range(40):

    # A) Check if any environments terminated
    terminated_envs = env.get_terminated_env_indices()
    if terminated_envs:
        print(f"   ⚠️  Environments {terminated_envs} terminated")
        reset_obs, reset_infos = env.reset_specific_envs(terminated_envs)
        for i, env_id in enumerate(terminated_envs):
            infos[env_id] = reset_infos[i] # reset previous info dict

    # Generate random actions for all environments
    actions = np.random.randint(0, 3, size=env.num_envs)

    print(f"\n   Step {step + 1}:")
    print(f"      Actions: {actions}")

    # Take step
    observations, rewards, terminateds, truncateds, infos = env.step(actions)

    print(f"      Rewards: {rewards}")
    print(f"      Terminated: {terminateds}")
    print(f"      Active environments: {env.get_active_count()}/{env.num_envs}")

    # Show filtered info (production mode)
    for i, info in enumerate(infos):
        print(f"      Env {i} info: {info}")

env.close()


   Step 1:
      Actions: [2 1 1 2]


2026-02-11 16:06:11,618 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/episode/vectorized_step "HTTP/1.1 200 OK"


      Rewards: [ 130.40225 -535.22565 -500.79715  136.0336 ]
      Terminated: [ True False False  True]
      Active environments: 2/4
      Env 0 info: {'step': 1, 'terminated': True, 'truncated': False}
      Env 1 info: {'step': 9, 'terminated': False, 'truncated': False}
      Env 2 info: {'step': 9, 'terminated': False, 'truncated': False}
      Env 3 info: {'step': 1, 'terminated': True, 'truncated': False}
   ⚠️  Environments [0, 3] terminated


2026-02-11 16:06:13,652 - httpx - INFO - HTTP Request: POST http://localhost:8001/api/v1/episode/vectorized_reset "HTTP/1.1 200 OK"



   Step 2:
      Actions: [0 1 2 0]


KeyboardInterrupt: 